In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
%matplotlib inline

import tensorflow as tf

In [ ]:
#Load Iris Dataset 
iris = load_iris()
X = iris.data
y = iris.target
target_names = iris.target_names
print(X.shape, y.shape)  #4 features
#Scale the data
scaler = MinMaxScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [ ]:
pd.DataFrame(iris.data,columns = iris.feature_names).head()

In [ ]:
def plot3clusters(X, title, vtitle):
  plt.figure()
  colors = ['navy', 'turquoise', 'darkorange']
  lw = 2

  for color, i, target_name in zip(colors, [0, 1, 2], target_names):
      plt.scatter(X[y == i, 0], X[y == i, 1], color=color, alpha=1., lw=lw,
                  label=target_name)
  plt.legend(loc='best', shadow=False, scatterpoints=1)
  plt.title(title)  
  plt.xlabel(vtitle + "1")
  plt.ylabel(vtitle + "2")
  plt.show()

In [ ]:
### edTest(test_check_ae) ###
#create an AE and fit it with our data using 2 neurons in the dense layer using keras' functional API
input_dim = X_scaled.shape[1]
encoding_dim = 2  

input_features = tf.keras.Input(shape=(input_dim,))
encoded = tf.keras.layers.Dense(encoding_dim, activation=None)(input_features)
decoded = tf.keras.layers.Dense(input_dim, activation=None)(encoded)

#create autoencoder model which has input as input_features and outputs decoded
autoencoder = tf.keras.Model(inputs = input_features, outputs=decoded)

autoencoder.compile(optimizer='adam', loss='mse')
print(autoencoder.summary())

#Notice the y in .fit() below - This takes a minute to run (output suppressed)
history = autoencoder.fit(X_scaled, X_scaled,
                epochs=1000,
                batch_size=16,
                shuffle=True,
                validation_split=0.1,
                verbose = 0)

#plot our loss 
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()



In [ ]:
preds = autoencoder.predict(X_scaled)
print(preds.shape)
plot3clusters(preds[:,:2], 'AutoEncoder - Decoded output 2 features', 'AE') 


In [ ]:
# use our encoded layer to encode the training input
encoder = tf.keras.Model(input_features, encoded)
encoded_data = encoder.predict(X_scaled)
print(encoded_data.shape)
plot3clusters(encoded_data, 'Encoded data', 'AE')  